# Some Tools to Generate FDS Input Lines

For example multiple `DEVC` lines.

In [ ]:
import os
import pylab
import string
import fdsreader
import matplotlib
import subprocess

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import Processing as proc

from importlib import reload


# Reload Python modules for changes that occured during runtime.
# reload(proc)

In [ ]:
print('Package Versions')
print('----------------')
print('Pandas version: {}'.format(pd.__version__))
print('Numpy version: {}'.format(np.__version__))
print('fdsreader version: {}'.format(fdsreader.__version__))
print('Matplotlib version: {}'.format(matplotlib.__version__))

In [ ]:
cwd = os.getcwd()
print(cwd)

In [ ]:
# Get letters for automatic labeling.
letters = list(string.ascii_uppercase)
print(string.ascii_uppercase)


# Path to experiment data.
exp_root = os.path.join("..", "GeneralInformation", "macfp-db")
exp_macfp_dir = os.path.join(exp_root, "Fire_Growth", 
                             "NIST_Parallel_Panel", "Experimental_Data")


# Directory used to collect the output produced by this notebook.
output_dir = "CreateFDSInputOutput"
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
    print("* Output directory created.")
else:
    print("* Output directory already exists.")

## Read Experimental Data

In [ ]:
# Get commit hash of MaCFP repo.
# ------------------------------------
# Commit/revision used here:
# MaCFP HEAD, revision (short): b8b0c1b
# MaCFP HEAD, revision  (long): b8b0c1b048f1a1901f2e012463f2a7c168eb486a
# MaCFP description: macfp-1.0-639-gb8b0c1b
# ------------------------------------

# Check where you are at.
# Short form of commit hash.
# (git rev-parse --short HEAD)
revision_s = subprocess.check_output(["git", "rev-parse", "--short", "HEAD"], 
                                     cwd=exp_root).strip().decode()

# Long form of commit hash.
# (git rev-parse HEAD)
revision_l = subprocess.check_output(["git", "rev-parse", "HEAD"], 
                                     cwd=exp_root).strip().decode()

# Tag (version) and commit number.
# Needs tags.
description = subprocess.check_output(["git", "describe"], 
                                     cwd=exp_root).strip().decode()


# Show actuals.
print("MaCFP git repo version")
print("------------------------------")
print(f"MaCFP HEAD, revision (short): {revision_s}")
print(f"MaCFP HEAD, revision  (long): {revision_l}")
print(f"MaCFP description: {description}")

In [ ]:
# Get file names.
os.listdir(exp_macfp_dir)

In [ ]:
# Read centre line heat flux to empty panel.
centreline_hf_path = os.path.join(exp_macfp_dir, "Burner_HF_Centerline_multi-layer.csv")
centreline_hf_df = pd.read_csv(centreline_hf_path, header=0, skiprows=[1])


# Check result.
centreline_hf_df.head()

### Burner RAMP

In [ ]:
# F-values for time steps. Copy over manually.
hrr_target = 60  # kW

# Initialise RAMP points.
burner_ramp_points = [[0,0, 0]]

for hrr_id, hrr in enumerate(centreline_hf_df.HRR):
    n_hrr = hrr / hrr_target
    n_time = centreline_hf_df.Time[hrr_id]
    
    burner_ramp_points.append([n_time, n_hrr, hrr])
    

burner_ramp_nucleus = "&RAMP ID='BurnerRamp', T={: >3}, F={} /"
for burner_ramp_point in burner_ramp_points:
    n_ramp = burner_ramp_nucleus.format(burner_ramp_point[0], 
                                        burner_ramp_point[1])
    print(n_ramp)

In [ ]:
# Check result.
plt.plot(np.array(burner_ramp_points)[:,0],
         np.array(burner_ramp_points)[:,2],
         marker='o')
    
    
# Plot meta data.
plt.xlabel("Time / s")
plt.ylabel("Energy Release Rate / kW")

plt.tight_layout()
# plt.legend()
plt.grid()


### Conductivity and Specific Heat RAMPs, Kaowool Blanket

Data is mixed from multiple sources. `MATL` and `SURF` are based on the FDS validation case ["NIST_NRC_Parallel_Panels"](https://github.com/firemodels/fds/blob/master/Validation/NIST_NRC_Parallel_Panels/FDS_Input_Files/PMMA_60_kW.fds). Thus, all parameters that are unknown are using the values of said case as default. For example the pea-gravel and the sand use the same `MATL`. Same is true for the Marinite and plywood boards.

#### Conductivity RAMP
Data from [MaCFP](https://github.com/MaCFP/matl-db/tree/master/PMMA/Validation_Data/NIST_Gasification_Apparatus) for high temperature. Data from [DBI/Lund](https://github.com/MaCFP/matl-db/tree/master/PMMA/Calibration_Data/DBI_Lund) for low temperature and density.

In [ ]:
# MaCFP: Morgan Thermal Ceramics Kaowool PM Insulation Board
macfp_kaowool = {
    "Density": 256, 
    "Heat Capacity": {
        "Temperatures": [980],
        "Values": [1.070]},
    "Conductivity":{
        "Temperatures": [260.0, 538.0, 816.0, 1093.0], 
        "Values": [0.0576, 0.085, 0.125, 0.183]}
}

# DBI/Lund: Morgan Thermal Ceramics Super wool plus
dbi_superwool = {
    "Density": 64, 
    "Heat Capacity": {
        "Temperatures": None,
        "Values": None},
    "Conductivity":{
        "Temperatures": [25.0, 30.0, 35.0, 40.0, 45.0, 50.0, 55.5, 60.0, 65.0, 70.0], 
        "Values": [0.03364, 0.03437, 0.03520, 0.03618, 0.03721, 0.03795, 0.03854, 0.03920, 0.04003, 0.04088]}
}

In [ ]:
# Initialise RAMP points.
cond_ramp_points = list()

# Collect data points for low temperature.
cond_data  = dbi_superwool["Conductivity"]
for temp_id, temp in enumerate(cond_data["Temperatures"]):
    val = cond_data["Values"][temp_id]
    cond_ramp_points.append([temp, val])

# Collect data points for heigh temperature.
cond_data  = macfp_kaowool["Conductivity"]
for temp_id, temp in enumerate(cond_data["Temperatures"]):
    val = cond_data["Values"][temp_id]
    cond_ramp_points.append([temp, val])
    
    
# Show result.
# cond_ramp_points

In [ ]:
# Create RAMP input.
ramp_lines = list()
ramp_nucleus = "&RAMP ID='COND_KAOWOOL_BLANKET', T={: >6}, F={} /"

for data_point in cond_ramp_points:
    ramp_line = ramp_nucleus.format(data_point[0],
                                    data_point[1])
    print(ramp_line)

In [ ]:
# Check result.
plt.plot(np.array(cond_ramp_points)[:,0],
         np.array(cond_ramp_points)[:,1],
         marker='o')
    
    
# Plot meta data.
plt.xlabel("Temperature / °C")
plt.ylabel("Thermal Conductivity / W/(m * K)")

plt.tight_layout()
# plt.legend()
plt.grid()


### Conductivity and Specific Heat RAMPs of the Panels

Data is mixed from multiple sources. `MATL` and `SURF` are based on the FDS validation case ["NIST_NRC_Parallel_Panels"](https://github.com/firemodels/fds/blob/master/Validation/NIST_NRC_Parallel_Panels/FDS_Input_Files/PMMA_60_kW.fds). Thus, all parameters that are unknown are using the values of said case as default. For example the Marinite and plywood boards use the same `MATL`.

#### Conductivity RAMP
Data from [MaCFP](https://github.com/MaCFP/macfp-db/tree/master/Fire_Growth/NIST_Parallel_Panel/Documentation).

### Path Length (RADI)

User guide:

For a given gas temperature and species composition, RadCal computes a single effective absorption coefficient
that is independent of wavelength. To calculate this coefficient, a user-specified `PATH_LENGTH` (m)
is needed. Its default value is 0.1 m. The choice of `PATH_LENGTH` can be based on the physical size of
the fire, the compartment, or the overall computational domain, depending on the application. The default
value has been chosen to capture accurately radiation heat transfer in and around the fire itself. A useful
“rule of thumb” for this length scale is 4V=A, where V is the volume of the region of interest and A is the
encompassing surface area. This region might be the volume occupied by the fire itself or a flashed-over compartment. Alternatively, if the application involves calculating the heat flux to distant targets, a more
appropriate `PATH_LENGTH` might be the distance from the fire to the target. A sensitivity analysis should be
done in any case to determine how the chosen `PATH_LENGTH` affects the predicted values.

In [ ]:
# Possible value for PATH_LENGTH.
domain_x = 0.8
domain_y = 1.2
domain_z = 3.2

domain_vol = domain_x * domain_y * domain_z

domain_surf = (domain_x * domain_y + \
               domain_x * domain_z + \
               domain_z * domain_y) * 2

path_length = 4 * domain_vol / domain_surf
path_length

### DEVC Locations

In [ ]:
# Device values.

# &DEVC ID='hf', XYZ=..., IOR=-2, QUANTITY='GAUGE HEAT FLUX', PROP_ID='hfp' /
# &PROP ID='hfp', GAUGE_TEMPERATURE=80., GAUGE_EMISSIVITY=0.9 /

# Defaults.
gauge_temperaturs = np.array([
    [20.0, 20.0, 20.0, 20.0, 20.0],
    [20.0, 20.0, 20.0, 20.0, 20.0],
    [20.0, 20.0, 20.0, 20.0, 20.0],
    [20.0, 20.0, 20.0, 20.0, 20.0]
])

gauge_emissivities = np.array([
    [1.0, 1.0, 1.0, 1.0, 1.0],
    [1.0, 1.0, 1.0, 1.0, 1.0],
    [1.0, 1.0, 1.0, 1.0, 1.0],
    [1.0, 1.0, 1.0, 1.0, 1.0]
])

In [ ]:
# Positions.
hf_gauge_locations = {
    "PanelFace": 15,
    "Heights": [20,50,75,100],
    "Horizontal": [-25,-15,0,15,25]}


comment_nucleus = "# Height: {} cm"
devc_nucleus = "\
&DEVC ID       = 'HF_y{}_z{}',\n\
      IOR      = -1,\n\
      XYZ      = {}, {}, {},\n\
      QUANTITY = 'GAUGE HEAT FLUX' /\n"



panelface = hf_gauge_locations["PanelFace"]
for height in hf_gauge_locations["Heights"]:
    print(comment_nucleus.format(height))
    
    for horizontal in hf_gauge_locations["Horizontal"]:
        x_pos = panelface/100
        y_pos = horizontal/100
        z_pos = height/100
        n_devc = devc_nucleus.format(horizontal,height,x_pos,y_pos,z_pos)
        
        print(n_devc)
    print('')  # Spacing